In [ ]:
import pandas as pd
import spotipy
#from fycharts.SpotifyCharts import SpotifyCharts
from spotipy.oauth2 import SpotifyClientCredentials

In [ ]:
df = pd.read_csv('global_2017-07-14--2017-07-21.csv')

In [ ]:
df

In [ ]:
df.columns = df.iloc[0]

In [ ]:
df = df.iloc[1:].reset_index()

In [ ]:
df.URL.astype('str')

In [ ]:
df.URL = df.URL.apply(lambda x : x[31:])

In [ ]:
df.rename(columns={'URL':'ID'}, inplace=True)

In [ ]:
df

In [ ]:
credentials = SpotifyClientCredentials(client_id="6b8119b6cec94f538151d6bb7fd99acd", client_secret="bb3cc9f8fdb04ddb9516ab01c7bfc257")
token = credentials.get_access_token()

spotify = spotipy.Spotify(auth=token)

features = spotify.audio_features(str(df['ID'].iloc[0]))


In [ ]:
features

In [ ]:
def extract_features(df, n_tracks = 50):
    dataframe = pd.DataFrame(columns=['id','danceability','energy','key','loudness','mode','speechiness',
                                     'acousticness','instrumentalness','liveness','valence','tempo','time_signature',
                                     'duration_ms'])
    for i in range(n_tracks):
        features = spotify.audio_features(str(df['ID'].iloc[i]))
        print(features[0]['danceability'])
        dataframe.append({'danceability' : features[0]['danceability'],
        'energy': features[0]['energy'],
        'key' : features[0]['key'],
        'loudness' : features[0]['loudness'],
        'mode' : features[0]['mode'],
        'speechiness' : features[0]['speechiness'],
        'acousticness' : features[0]['acousticness'],
        'instrumentalness' : features[0]['instrumentalness'],
        'liveness' : features[0]['liveness'],
        'valence' : features[0]['valence'],
        'tempo' : features[0]['tempo'],
        'time_signature' : features[0]['time_signature'],
        'duration_ms' : features[0]['duration_ms']}, ignore_index=True)
    return dataframe

In [ ]:
def extract_features(df, n_tracks = 50):
    dataframe = pd.DataFrame(columns=['id','danceability','energy','key','loudness','mode','speechiness',
                                     'acousticness','instrumentalness','liveness','valence','tempo','time_signature',
                                     'duration_ms'])
    for i in range(n_tracks):
        features = spotify.audio_features(str(df['ID'].iloc[i]))
        dataframe.loc[i] = [ features[0]['id'],
        features[0]['danceability'],
        features[0]['energy'],
        features[0]['key'],
        features[0]['loudness'],
        features[0]['mode'],
        features[0]['speechiness'],
        features[0]['acousticness'],
        features[0]['instrumentalness'],
        features[0]['liveness'],
        features[0]['valence'],
        features[0]['tempo'],
        features[0]['time_signature'],
        features[0]['duration_ms']]
    return dataframe

In [ ]:
prova = extract_features(df, n_tracks=200)

In [ ]:
prova

In [ ]:
output = df.join(prova.set_index('id'), on='ID')

In [ ]:
output.to_csv('global_08-2017.csv')

In [ ]:
output

test=output.iloc[9]
print(test)
print(type(test['Track Name']))
if type(test['Track Name']) != float and type(test['Artist']) != float:
    print('no empty')
else: 
    print('empty')

## Testing iTunes API for genre labeling
Simple test of quering on iTunes API

In [ ]:
import requests
import json
entry = 'ghali'
a = 'https://itunes.apple.com/search?term='+entry+'.&limit=1'
b = requests.get(a).json()
print(b['results'][0]['primaryGenreName'])
print(json.dumps(b, indent=2))

# Automatizing the feature extraction process

In [ ]:
import pandas as pd
import spotipy
#from fycharts.SpotifyCharts import SpotifyCharts
from spotipy.oauth2 import SpotifyClientCredentials
import os
import requests
import json
import time

In [ ]:
def extract_features(df, n_tracks = 50):
    dataframe = pd.DataFrame(columns=['id','danceability','energy','key','loudness','mode','speechiness',
                                     'acousticness','instrumentalness','liveness','valence','tempo','time_signature',
                                     'duration_ms'])
    for i in range(n_tracks):
        features = spotify.audio_features(str(df['ID'].iloc[i]))
        print(features[0]['danceability'])
        dataframe.append({'danceability' : features[0]['danceability'],
        'energy': features[0]['energy'],
        'key' : features[0]['key'],
        'loudness' : features[0]['loudness'],
        'mode' : features[0]['mode'],
        'speechiness' : features[0]['speechiness'],
        'acousticness' : features[0]['acousticness'],
        'instrumentalness' : features[0]['instrumentalness'],
        'liveness' : features[0]['liveness'],
        'valence' : features[0]['valence'],
        'tempo' : features[0]['tempo'],
        'time_signature' : features[0]['time_signature'],
        'duration_ms' : features[0]['duration_ms']}, ignore_index=True)
    return dataframe

In [ ]:
def extract_features(df, n_tracks = 50):
    dataframe = pd.DataFrame(columns=['id','danceability','energy','key','loudness','mode','speechiness',
                                     'acousticness','instrumentalness','liveness','valence','tempo','time_signature',
                                     'duration_ms'])
    for i in range(n_tracks):
        features = spotify.audio_features(str(df['ID'].iloc[i]))
        dataframe.loc[i] = [ features[0]['id'],
        features[0]['danceability'],
        features[0]['energy'],
        features[0]['key'],
        features[0]['loudness'],
        features[0]['mode'],
        features[0]['speechiness'],
        features[0]['acousticness'],
        features[0]['instrumentalness'],
        features[0]['liveness'],
        features[0]['valence'],
        features[0]['tempo'],
        features[0]['time_signature'],
        features[0]['duration_ms']]
    return dataframe

In [ ]:
#checking data integrity. Checks for missing fields in Artist name. Return the rows with missing values + the file name. 
#These data are missing because of copyright issues. They changed the author of the song/removed the song. 
import numpy as np

from IPython.display import display, HTML

path = os.getcwd()
file_folder = os.path.join(path,'br')

for csv_file in os.listdir(file_folder):
    file = os.path.join(file_folder,csv_file)
    #print(csv_file)
    df = pd.read_csv(file)
    #display(df)
    df.columns = df.iloc[0]
    df = df.iloc[1:].reset_index()
    df.URL.astype('str')
    df.URL = df.URL.apply(lambda x : x[31:])
    df.rename(columns={'URL':'ID'}, inplace=True)
    
    res = df.loc[df['Artist'].isnull(), 'index'].tolist()
    if len(res) != 0:
        print(res)
        print(csv_file)

    
    
 

In [ ]:
path = os.getcwd()
file_folder = os.path.join(path,'br 2')
save_folder_path = os.path.join(path, "br_features")
#os.mkdir(save_folder_path)

for csv_file in os.listdir(file_folder): 
    print(csv_file)
    file = os.path.join(file_folder,csv_file)
    df = pd.read_csv(file)
    df.columns = df.iloc[0]
    df = df.iloc[1:].reset_index()
    df.URL.astype('str')
    df.URL = df.URL.apply(lambda x : x[31:])
    df.rename(columns={'URL':'ID'}, inplace=True)
    credentials = SpotifyClientCredentials(client_id="6b8119b6cec94f538151d6bb7fd99acd", client_secret="bb3cc9f8fdb04ddb9516ab01c7bfc257")
    token = credentials.get_access_token()
    spotify = spotipy.Spotify(auth=token)
    features = spotify.audio_features(str(df['ID'].iloc[0]))
    prova = extract_features(df, n_tracks=200)
    output = df.join(prova.set_index('id'), on='ID')
    output['genre']= "none"
    for index, row in output.iterrows():
        artist = row["Artist"]
        song = row["Track Name"]
        if type(artist) != float and type(song) != float:
            song = song.split('(')[0]
            query_entry = str(artist)+" "+str(song)
            query_entry = query_entry.replace("with", "")
            query_entry= query_entry.replace('Remastered', "")
            query_entry= query_entry.replace('Version', "")

            query_entry = query_entry.replace("'", "")
            query_entry = query_entry.replace('"', "")

            query_entry=query_entry.replace(" ", "+")

            #query_entry = query_entry[0:60]
            query_entry = query_entry.lower()
            print(query_entry)
            a = 'https://itunes.apple.com/search?term='+query_entry+'&limit=1'
            test = requests.get(a)
            print(test)
            b = test.json()
            if b['resultCount'] == 1:

                print(b['results'][0]['primaryGenreName'])
                #assign genre to song
                print(index)
                output.iloc[index, output.columns.get_loc('genre')]= b['results'][0]['primaryGenreName']
            time.sleep(0.5)
            #print(json.dumps(b, indent=2))
    output.to_csv(os.path.join(save_folder_path,csv_file))




In [ ]:
#Testing on spotify release date.
import os
import pandas as pd
from IPython.display import display, HTML

credentials = SpotifyClientCredentials(client_id="6b8119b6cec94f538151d6bb7fd99acd", client_secret="bb3cc9f8fdb04ddb9516ab01c7bfc257")
token = credentials.get_access_token()
spotify = spotipy.Spotify(auth=token)
#features = spotify.audio_features(str(df['ID'].iloc[0]))

path = os.getcwd()
file_folder = os.path.join(path,'it_features')
for csv_file in os.listdir(file_folder):
    csv_path = os.path.join(file_folder,csv_file)
    df_temp = pd.read_csv(csv_path)
    df_temp['release date'] = 'none'
    for index, row in df_temp.iterrows():
        ID = row["ID"]
        result = spotify.track(ID)
        release_date=result['album']['release_date']
        df_temp.iloc[index, df_temp.columns.get_loc('release date')] = release_date
    df_temp.to_csv(os.path.join(file_folder,csv_file))

In [ ]:
credentials = SpotifyClientCredentials(client_id="6b8119b6cec94f538151d6bb7fd99acd", client_secret="bb3cc9f8fdb04ddb9516ab01c7bfc257")
token = credentials.get_access_token()
spotify = spotipy.Spotify(auth=token)
result = spotify.track('6GpvHaqp0Yybx9P6YnodRD')
print(result)